## Opens a basemap in image location and prints list of coordinates selected by user
**Idea is to open images from cluster on top of this map, but not sure how to assign local images URL that works with Jupyter on a remote server (i.e. LocalTileServer can't work in this case) Due to this being hosted on a remote server, we are also currently contrained to Python 3.6, so newer tools like geemap don't work**

## Imports

In [ ]:
import os
import sys
from pathlib import Path
import datetime

import rasterio as rio
from rasterio import plot
from rasterio.plot import show
from rasterio.warp import calculate_default_transform, reproject, Resampling
import matplotlib.pyplot as plt
import shutil
import tempfile
import json
import random
import numpy as np
import pandas as pd
import geopandas as gpd
import pyproj
from pyproj import Proj, transform
#import cartopy.crs as ccrs
from shapely.geometry import box
from shapely.geometry import shape
from shapely.geometry import MultiPoint
from shapely.geometry import Point
from shapely.geometry import Polygon
import xarray as xr
import base64
from PIL import Image
import csv

from ipywidgets import Label
from ipyleaflet  import Map, GeoData, basemaps, LayersControl, ImageOverlay
import xarray_leaflet
#from localtileserver import get_leaflet_tile_layer, TileClient  ### Note: This doesn't work on remote server

pyver = float((sys.version)[:3])
if pyver >= 3.8:
    import geemap   ##Note geemap doesn't work with Python 3.6 which is native on cluster, but works in a conda envt

%matplotlib inline

In [ ]:
sys.path.append(r"../LUCinSA_helpers")
from file_info import *
from plot import *

In [ ]:
'''
PARAMETERS: modify in Notebook_settings notebook, then run that notebook and this cell to update here
DO not modify this cell
'''

%store -r basic_config
print("Basic Parameters: \n PrintDate = {} \n brdf_dir = {} \n grid_cell = {} \n index_dir = {} \n out_dir = {}"
      .format(basic_config['today'], basic_config['brdf_dir'],basic_config['grid_cell'],basic_config['index_dir'],basic_config['out_dir']))

%store -r single_plot_params
print("Plotting Parameters: \n plot_yr = {} \n plot_day = {} \n viewband = {} \n image_type = {}"
      .format(single_plot_params['plot_yr'],single_plot_params['plot_day'],single_plot_params['viewband'],single_plot_params['image_type']))

%store -r interactive_plot_params
print( "Shapefile = {} \n If point, file is {} \n If poly, file is {} \n input crs = {}"
     .format (interactive_plot_params['shpfile'], interactive_plot_params['ptfile'],interactive_plot_params['polyfile'],interactive_plot_params['inputCRS']))

In [ ]:
## Get a sample image of type image_type, closest to the date specified with plot_day and plot_yr parameters
if single_plot_params['image_type'] == 'Smooth':
    samp_img = get_closest_image(basic_config['index_dir'],single_plot_params['image_type'],basic_config['data_source'],single_plot_params['plot_yr'],single_plot_params['plot_day'])
else:
    samp_img = get_closest_image(basic_config['brdf_dir'],single_plot_params['image_type'],basic_config['data_source'],single_plot_params['plot_yr'],single_plot_params['plot_day'])

print(samp_img)

In [ ]:
coord_list = []

def handle_interaction(**kwargs):
    if kwargs.get('type') == 'click':
        label.value = str(kwargs.get('coordinates'))
        coords =eval(label.value) 
        coord_list.append(coords)
        print(coord_list)

if samp_img.endswith('.tif'):
    with rio.open(samp_img) as src:
        tsamp = src.read()
        print('original image is in {}'.format(src.crs))
        
        dst_crs = 'EPSG:4326'
        kwargs = src.meta.copy()
        transform, width, height = calculate_default_transform(src.crs, dst_crs, src.width, src.height, *src.bounds)
        kwargs.update({'crs': dst_crs,'transform': transform,'width': width,'height': height})
        samp_img_ll = (os.path.join(basic_config['out_dir'],os.path.basename(samp_img)))
        with rio.open(samp_img_ll, 'w+', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rio.band(src, i),
                    destination=rio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest)
    with rio.open(aamp_img_ll) as src_ll:
        #tsampll = src_ll.read()
        print('new image is in {}'.format(src_ll.crs))
        
        img_centerT = src_ll.xy(src_ll.height // 2, src_ll.width // 2)
        img_center = [img_centerT[1],img_centerT[0]]
        sw = [src_ll.bounds[1],src_ll.bounds[0]]
        ne = [src_ll.bounds[3],src_ll.bounds[2]]
        img_bounds = [SW, NE]
        print('Image center is at: {}'.format(img_center))
        print('SW and NW corners are at: {}'.format(img_bounds))
    
    #with open(SampImg_ll, "rb") as file:
        #base64_encoded = base64.b64encode(file.read())
    
    ###ipyleaflet cannot render .tif files. Convert to .png 
    samp_png = os.path.splitext(os.path.join(basic_config['out_dir'],os.path.basename(samp_img)))[0] + ".png"
    im = Image.open(samp_img_ll)
    im.thumbnail(im.size)
    im.save(samp_png, "png", quality=100)
    
    relative_path = os.path.join('../',samp_png)
                   
elif samp_img.endswith('.nc'):
    xrimg = xr.open_dataset(samp_img)
    xrcrs = xrimg.crs
    print(xrcrs)
     
    source_crs = interactive_plot_params['inputCRS']
    target_crs = 'epsg:4326' # Global lat-lon coordinate system (ipyleaflet only uses lat lon)
    crs_to_latlon = pyproj.Transformer.from_crs(source_crs, target_crs, always_xy=True)

    #find the new bounds in lat lon
    lonB, latB = crs_to_latlon.transform([xrimg.x.min(), xrimg.x.max()],[xrimg.y.min(), xrimg.y.max()])
    img_bounds = [(latB[0],lonB[0]),(latB[1],lonB[1])]
    img_center = (latB[0]+((latB[1]-latB[0])/2), lonB[0]+((lonB[1]-lonB[0])/2))
    
    ##reproject to Lat Lon, but TODO: need to reduce XY dimensions first
    #X, Y = np.meshgrid(xrimg.x, xrimg.y)
    #lat1, lon1 = albers_to_latlon.transform(X, Y)
    #lon1 = lon1.reshape(X.shape)
    #lat1 = lat1.reshape(Y.shape)
    #xrimg.coords['lat'] = (xrimg.dims, lat)
    #xrimg.coords['lon'] = (xrimg.dims, lon)
    
    #img_gdf = gpd.GeoDataFrame({"geometry":[img_poly],"crs":"EPSG:4326"})
    #bounds_layer = GeoData(geo_dataframe=img_gdf, name='sampImg')
    
    #relativePath = ##TODO: make this point to the layer to add to map
    
    ###TODO: Add methods to view multilayer composite (take methods from 1b)

else:
    print('only set up to read .tif and .nc files at the moment')

m1 = Map(center=img_center, zoom=12, basemap=basemaps.Esri.WorldImagery)

###TODO: Fix this to show sample image (need a url for a local file that works with Jupyter on a remote server...)
#image = ImageOverlay(url=relativePath, bounds=img_bounds)
#m1.add_layer(image);

### To enable display of coordinates on click:
label = Label()
display(label)
m1.on_interaction(handle_interaction)

### TO Add A shapefile to map (optional):
if interactive_plot_params['shpfile'] != None:
    if interactive_plot_params['shpfile'] == 'point':
        if os.path.basename(interactive_plot_params['ptfile']).endswith('.txt'):
            ptsdf = pd.read_table(interactive_plot_params['ptfile'], index_col=0, sep=",")
            shps = gpd.GeoDataFrame(ptsdf,geometry=gpd.points_from_xy(ptsdf.XCoord,ptsdf.YCoord),crs=interactive_plot_params['inputCRS'])
        elif os.path.basename(interactive_plot_params['ptfile']).endswith('.csv'):
            ptsdf = pd.read_csv(interactive_plot_params['ptfile'], index_col=0)
            shps = gpd.GeoDataFrame(ptsdf,geometry=gpd.points_from_xy(ptsdf.XCoord,ptsdf.YCoord),crs=interactive_plot_params['inputCRS'])
        else:
            shps = gpd.read_file(interactive_plot_params['ptfile'])
    elif interactive_plot_params['shpfile'] == 'poly':
        shps = gpd.read_file(interactive_plot_params['polyfile'])
    shps_ll = shps.to_crs("EPSG:4326")
    viewshp = GeoData(geo_dataframe = shps_ll)
    m1.add_layer(viewshp)

m1

In [ ]:
### Convert list of coordinates back to original CRS and print to file:
coord_listX = []
coord_listY = []
transformer = pyproj.Transformer.from_crs("epsg:4326", interactive_plot_params['inputCRS'])
for pt in transformer.itransform(coord_list): 
    print('{:.3f} {:.3f}'.format(pt[0],pt[1]))
    coord_listX.append(pt[0])
    coord_listY.append(pt[1])
    coords = {'XCoord':coord_listX,'YCoord':coord_listY}
coorddb = pd.DataFrame(coords)
coorddb = coorddb.astype({'XCoord':'float','YCoord':'float'})
pd.DataFrame(coorddb).to_csv(os.path.join(basic_config['out_dir'],'SelectedCoords.csv'), sep=',', na_rep='NaN', index=True)
print(coorddb)

## Get Values at Coordinates
###TODO: Make this work with any image type (need to clean methods throughout; not L1C & TS but smooth and raw)

In [ ]:
if isinstance(coorddb, pd.DataFrame):
    ptsdf = coorddb
else:
    PtFile=os.path.join(basicConfig['out_dir'],'SelectedCoords.csv')
    ptsdf = pd.read_csv(PtFile)
### pt file is in SA Albers Equal Area Conic (ESRI:102033)

pts = gpd.GeoDataFrame(ptsdf,geometry=gpd.points_from_xy(ptsdf.XCoord,ptsdf.YCoord),crs='esri:102033')
xy = [pts['geometry'].x, pts['geometry'].y]
coords = list(map(list, zip(*xy)))
    
if SinglePlotParams['imageType'] == 'TS':
    img_name = os.path.basename(SampImg)[:7]
    with rasterio.open(SampImg, 'r') as src:
        ptsval = [sample[0] for sample in src.sample(coords)]

elif SinglePlotParams['imageType'] == 'Sentinel' or SinglePlotParams['imageType'] == 'Landsat' or SinglePlotParams['imageType'] == 'All' :
    if os.path.basename(SampImg).startswith('L1C'):
        YYYY = int(os.path.basename(SampImg)[19:23])
        MM = int(os.path.basename(SampImg)[23:25])
        DD = int(os.path.basename(SampImg)[25:27])
    elif os.path.basename(SampImg).startswith('LC') or os.path.basename(SampImg).startswith('LT') or os.path.basename(SampImg).startswith('LE'):
        YYYY = int(os.path.basename(SampImg)[17:21])
        MM = int(os.path.basename(SampImg)[21:23])
        DD = int(os.path.basename(SampImg)[23:25])
ymd = datetime.datetime(YYYY, MM, DD)
doy = ymd.strftime('%j')
img_name = str(YYYY)+doy
xrimg = xr.open_dataset(SampImg)
xr_val = xrimg[SinglePlotParams['viewBand']].where(xrimg[SinglePlotParams['viewBand']] < 10000)

vals=[]
for index, row in pts.iterrows():
    thispt_val = xr_val.sel(x=pts['geometry'].x[index],y=pts['geometry'].y[index], method='nearest', tolerance=30)
    this_val = thispt_val.values
    vals.append(this_val)
pts[SinglePlotParams['Viewband']] = vals
print(pts)

In [ ]:
if basicConfig['spec_index'] == 'evi2':
    index_val = 2.5 * ((nir_val - b2_val) / (nir_val + 1.0 + 2.4 * b2_val))

## To save an html copy of this notebook with all outputs:

In [ ]:
### Run to print output as html
outName = str(basicConfig['country']+'1c_InteractiveMapSession'
!jupyter nbconvert --output-dir='./Outputs' --to html --no-input --output=$outName 1c_ExploreData_InteractiveMap.ipynb